In [ ]:
import akshare as ak
import pandas as pd
#import datetime
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#concepts_list = ak.stock_board_concept_name_ths()

concepts_list = pd.read_csv('./static/update_concepts_list.csv',encoding="gbk")

In [ ]:
concepts_list['概念名称'] = concepts_list['概念名称'].str.replace('概念股','').str.replace('概念','').str.replace(' ','')

# check concepts_list updates
# new_concepts = set(concepts_list['概念名称']).difference(concepts_checklist['概念名称'])
# out_concepts = set(concepts_checklist['概念名称']).difference(concepts_list['概念名称'])
# print( out_concepts )

global stock_concepts
stock_concepts = pd.DataFrame()


def get_concept_components(concept):
    global stock_concepts
    print('正在获取同花顺概念成分股： ' + concept['概念名称'], str(row+1) + '/'+str(len(concepts_list)))
    stock_board_cons_df = ak.stock_board_cons_ths(symbol=concept['代码'])
    stock_board_cons_df['概念名称'] = concept['概念名称']
    stock_board_cons_df['概念代码'] = concept['代码']
    stock_concepts = stock_concepts.append(stock_board_cons_df)


import threading
threads = []


for row, concept in concepts_list.head(2).iterrows():
    t = threading.Thread(target = get_concept_components, args = (concept,))
    threads.append(t)

print(threads)

for t in threads: 
    t.start()

# 等待所有thread完成之后再执行之后的代码    
for t in threads: 
    t.join()

In [ ]:
stock_concepts = stock_concepts.groupby(['代码','名称'])['概念名称'].apply(lambda x: ",".join \
                            (list(set(x.str.cat(sep=',').split(','))))).reset_index()

In [ ]:
stock_concepts['行业概念'] = ''
stock_concepts['事件概念'] = ''
stock_concepts['交易概念'] = ''


# 将同花顺概念分为 “行业”，“事件”和“交易”三类
concepts_checklist = pd.read_csv('./static/concepts_list.csv',encoding="gbk") # list got from old iwencai api / more comple but outdate
industry_concepts_list = concepts_checklist.loc[concepts_checklist['分类'].isin(['行业'])]['概念名称'].values.tolist()
events_concepts_list = concepts_checklist.loc[concepts_checklist['分类'].isin(['事件','知名企业','农村','改革','军工','国企体系'])]['概念名称'].values.tolist()
trading_concepts_list = concepts_checklist.loc[concepts_checklist['分类'].isin(['交易'])]['概念名称'].values.tolist() # 区域missing


for row, l in stock_concepts.iterrows():
    stock_concepts['行业概念'][row] = ','.join(set(l['概念名称'].split(',')).intersection(industry_concepts_list))
    stock_concepts['事件概念'][row] = ','.join(set(l['概念名称'].split(',')).intersection(events_concepts_list))
    stock_concepts['交易概念'][row] = ','.join(set(l['概念名称'].split(',')).intersection(trading_concepts_list))



In [ ]:
stock_concepts

In [ ]:
ths_concepts['概念名称'] = ths_concepts['概念名称'].str.replace('概念股','').str.replace('概念','')
ths_concepts

In [ ]:
ths_industry =  ak.stock_board_industry_name_ths() 

In [ ]:
ths_industry

In [ ]:
ak.stock_zh_a_spot_em()

In [ ]:
stock_zh_a_tick_tx_df = ak.stock_zh_a_tick_tx_js(symbol="sh688295")
stock_zh_a_tick_tx_df

In [ ]:
stock_zh_a_tick_tx_df.to_csv('trades.csv')

In [ ]:
import pandas as pd


def Categorize_Concept_ths():
    
    # 所有概念group
    global all_concepts_grouped_ths
    global industry_concepts_grouped_ths
    global events_concepts_grouped_ths
    global trading_concepts_grouped_ths

    all_concepts_grouped_ths = all_stocks_concepts_ths.groupby(['代码','名称'])['概念名称_ths'].apply(lambda x: ",".join \
                                (list(set(x.str.cat(sep=',').split(','))))).reset_index()
 
    # 将同花顺概念分为 “行业”，“事件”和“交易”三类
    industry_concepts_list_ths = concepts_ths.loc[concepts_ths['概念种类'].isin(['行业'])]['概念_ths'].values.tolist()
    events_concepts_list_ths = concepts_ths.loc[concepts_ths['概念种类'].isin(['事件','区域','知名企业'])]['概念_ths'].values.tolist()
    trading_concepts_list_ths = concepts_ths.loc[concepts_ths['概念种类'].isin(['金融'])]['概念_ths'].values.tolist()
 
    # 三类概念分别group
    industry_concepts_grouped_ths = all_stocks_concepts_ths.loc[all_stocks_concepts_ths['概念名称_ths'].isin(industry_concepts_list_ths)].\
                                groupby(['代码','名称'])['概念名称_ths'].apply(lambda x: ",".join \
                                (list(set(x.str.cat(sep=',').split(','))))).reset_index().rename(columns={'概念名称_ths': '行业概念_ths'})

    events_concepts_grouped_ths = all_stocks_concepts_ths.loc[all_stocks_concepts_ths['概念名称_ths'].isin(events_concepts_list_ths)].\
                                groupby(['代码','名称'])['概念名称_ths'].apply(lambda x: ",".join \
                                (list(set(x.str.cat(sep=',').split(','))))).reset_index().rename(columns={'概念名称_ths': '事件概念_ths'})

    trading_concepts_grouped_ths = all_stocks_concepts_ths.loc[all_stocks_concepts_ths['概念名称_ths'].isin(trading_concepts_list_ths)].\
                                groupby(['代码','名称'])['概念名称_ths'].apply(lambda x: ",".join \
                                (list(set(x.str.cat(sep=',').split(','))))).reset_index().rename(columns={'概念名称_ths': '交易概念_ths'})


def Categorize_Concept_em():

    global all_concepts_grouped_em
    global industry_concepts_grouped_em
    global events_concepts_grouped_em
    global trading_concepts_grouped_em

     #所有概念表格
    all_concepts_grouped_em = all_stocks_concepts_em.groupby(['代码','名称'])['概念名称_em'].apply(lambda x: ",".join \
                                (list(set(x.str.cat(sep=',').split(','))))).reset_index()
 
    #将东方财富概念分为 “行业”，“事件”和“交易”三类
    industry_concepts_list_em = concepts_em.loc[concepts_em['概念种类'].isin(['行业'])]['概念_em'].values.tolist()
    events_concepts_list_em = concepts_em.loc[concepts_em['概念种类'].isin(['事件','区域','知名企业'])]['概念_em'].values.tolist()
    trading_concepts_list_em = concepts_em.loc[concepts_em['概念种类'].isin(['金融'])]['概念_em'].values.tolist()

    #分别制作三类概念数据表格
    industry_concepts_grouped_em = all_stocks_concepts_em.loc[all_stocks_concepts_em['概念名称_em'].isin(industry_concepts_list_em)].\
                                groupby(['代码','名称'])['概念名称_em'].apply(lambda x: ",".join \
                                (list(set(x.str.cat(sep=',').split(','))))).reset_index().rename(columns={'概念名称_em': '行业概念_em'})

    events_concepts_grouped_em = all_stocks_concepts_em.loc[all_stocks_concepts_em['概念名称_em'].isin(events_concepts_list_em)].\
                                groupby(['代码','名称'])['概念名称_em'].apply(lambda x: ",".join \
                                (list(set(x.str.cat(sep=',').split(','))))).reset_index().rename(columns={'概念名称_em': '事件概念_em'})

    trading_concepts_grouped_em = all_stocks_concepts_em.loc[all_stocks_concepts_em['概念名称_em'].isin(trading_concepts_list_em)].\
                                groupby(['代码','名称'])['概念名称_em'].apply(lambda x: ",".join \
                                (list(set(x.str.cat(sep=',').split(','))))).reset_index().rename(columns={'概念名称_em': '交易概念_em'})






read_route = './ungrouped_data/'
save_route = './grouped_data/'

concepts_ths = pd.read_csv('./行业概念/ths_概念.csv',encoding="gbk")
concepts_em = pd.read_csv('./行业概念/em_概念.csv',encoding="gbk") # 经过优化后的概念列表

all_stocks_industry_ths = pd.read_csv('./ungrouped_data/all_stocks_industry_ths.csv',encoding="gb2312")
print(all_stocks_industry_ths)
all_stocks_concepts_ths = pd.read_csv('./ungrouped_data/all_stocks_concepts_ths_ungrouped.csv',encoding="gb2312")
all_stocks_concepts_em  = pd.read_csv('./ungrouped_data/all_stocks_concepts_em_ungrouped.csv' ,encoding="gb2312")

ts_stocks_info = pd.read_csv('./ungrouped_data/ts_stocks_info.csv',encoding="gbk",encoding_errors='ignore')
stock_zcfz_em  = pd.read_csv('./ungrouped_data/balance_sheet.csv',encoding="gbk",encoding_errors='ignore')
stock_lrb_em   = pd.read_csv('./ungrouped_data/revenue.csv',encoding="gbk",encoding_errors='ignore')


Categorize_Concept_ths()
Categorize_Concept_em()

# # merge 行业和所有概念信息
all_stocks_info = all_stocks_industry_ths.merge(all_concepts_grouped_ths,how='outer',left_on=['代码','名称'],right_on=['代码','名称']) \
                                                .merge(industry_concepts_grouped_ths,how='outer',left_on=['代码','名称'],right_on=['代码','名称']) \
                                                .merge(events_concepts_grouped_ths,how='outer',left_on=['代码','名称'],right_on=['代码','名称']) \
                                                .merge(trading_concepts_grouped_ths,how='outer',left_on=['代码','名称'],right_on=['代码','名称']) \
                                                .merge(stock_lrb_em,how='outer',left_on=['代码','名称'],right_on=['股票代码','股票简称']) \
                                                .merge(stock_zcfz_em,how='outer',left_on=['代码','名称'],right_on=['股票代码','股票简称']) \
                                                .merge(ts_stocks_info,how='outer',left_on=['代码'],right_on=['代码']) 

all_stocks_info.to_csv( save_route + 'all_stocks_info.csv')


all_stocks_info_main = all_stocks_info[['代码','名称','流通市值','市盈率','行业名称_ths','概念名称_ths','行业概念_ths','事件概念_ths','交易概念_ths', \
                                    '概念名称_em','行业概念_em','事件概念_em','交易概念_em','公司介绍','营业范围','主营业务','注册资本','省份','城市', \
                                    '净利润','净利润同比','营业总收入','营业总收入同比','资产负债率','股东权益合计']]
all_stocks_info_main.to_csv( save_route + '股票行业概念信息.csv')

In [ ]:
stock_zh_a_spot_em_df = ak.stock_zh_a_spot_em()
stock_zh_a_spot_em_df.to

In [ ]:
stock_zh_a_spot_em_df.to_csv('data.csv')

In [ ]:
from datetime import date
today = date.today()

In [ ]:
#同花顺概念列表需要到网站上爬取  http://q.10jqka.com.cn/gn/， 后续考虑写一个爬虫

#后续考虑自动化更新概念列表及概念名称统一
# ----------------------------同花顺----------------------
# 硅能源	碳化硅
# 共享单车	共享经济
# 债转股(AMC)	债转股
# 东数西算（算力）	东数西算




#东方财富网概念列表可以通过代码获得
# stock_board_concept_name_em_df = ak.stock_board_concept_name_em()[['板块名称','板块代码']]

In [ ]:
#获取股票同花顺行业信息
industries_ths = pd.read_csv('./行业概念/ths_行业.csv',encoding="gbk")
all_stocks_industry_ths = pd.DataFrame()
for row, industry in industries_ths.iterrows():
    print('正在获取行业成分股： ' + industry['行业名称_ths'], str(row+1) + '/'+str(len(industries_ths)) )
    stock_board_cons_ths_df = ak.stock_board_cons_ths(symbol=industry['行业代码_ths'])
    stock_board_cons_ths_df['行业名称_ths'] = industry['行业名称_ths']
    stock_board_cons_ths_df['行业代码_ths'] = industry['行业代码_ths']
    all_stocks_industry_ths = all_stocks_industry_ths.append(stock_board_cons_ths_df)

# all_stocks_industry_ths.to_csv('all_stocks_industry_ths.csv')
# all_stocks_industry_ths

In [ ]:
#待解决： 断点续传问题
# 获取同花顺概念信息
concepts_ths = pd.read_csv('./行业概念/ths_概念.csv',encoding="gbk")
all_stocks_concepts_ths = pd.DataFrame()
for row, concept in concepts_ths.iterrows():
    print('正在获取概念成分股： ' + concept['概念_ths'], str(row+1) + '/'+str(len(concepts_ths)))
    stock_board_cons_ths_df = ak.stock_board_cons_ths(symbol=concept['概念代码_ths'])
    stock_board_cons_ths_df['概念名称_ths'] = concept['概念_ths']
    stock_board_cons_ths_df['概念代码_ths'] = concept['概念代码_ths']
    all_stocks_concepts_ths = all_stocks_concepts_ths.append(stock_board_cons_ths_df)
# all_stocks_concepts_ths.to_csv('all_stocks_concepts_ths.csv')
# all_stocks_concepts_ths


# 所有概念group
all_concepts_stocks_ths = all_stocks_concepts_ths.groupby(['代码','名称'])['概念名称_ths'].apply(lambda x: ",".join \
                            (list(set(x.str.cat(sep=',').split(','))))).reset_index()


# 将同花顺概念分为 “行业”，“事件”和“交易”三类
industry_concepts_list_ths = concepts_ths.loc[concepts_ths['概念种类'].isin(['行业'])]['概念_ths'].values.tolist()
events_concepts_list_ths = concepts_ths.loc[concepts_ths['概念种类'].isin(['事件','区域','知名企业'])]['概念_ths'].values.tolist()
trading_concepts_list_ths = concepts_ths.loc[concepts_ths['概念种类'].isin(['金融'])]['概念_ths'].values.tolist()

# 三类概念分别group
industry_concepts_stocks_ths = all_stocks_concepts_ths.loc[all_stocks_concepts_ths['概念名称_ths'].isin(industry_concepts_list_ths)]
industry_concepts_stocks_ths = industry_concepts_stocks_ths.groupby(['代码','名称'])['概念名称_ths'].apply(lambda x: ",".join \
                            (list(set(x.str.cat(sep=',').split(','))))).reset_index().rename(columns={'概念名称_ths': '行业概念_ths'})

events_concepts_stocks_ths = all_stocks_concepts_ths.loc[all_stocks_concepts_ths['概念名称_ths'].isin(events_concepts_list_ths)]
events_concepts_stocks_ths = events_concepts_stocks_ths.groupby(['代码','名称'])['概念名称_ths'].apply(lambda x: ",".join \
                            (list(set(x.str.cat(sep=',').split(','))))).reset_index().rename(columns={'概念名称_ths': '事件概念_ths'})

trading_concepts_stocks_ths = all_stocks_concepts_ths.loc[all_stocks_concepts_ths['概念名称_ths'].isin(trading_concepts_list_ths)]
trading_concepts_stocks_ths = trading_concepts_stocks_ths.groupby(['代码','名称'])['概念名称_ths'].apply(lambda x: ",".join \
                            (list(set(x.str.cat(sep=',').split(','))))).reset_index().rename(columns={'概念名称_ths': '交易概念_ths'})

In [ ]:
concepts_em = pd.read_csv('./行业概念/em_概念.csv',encoding="gbk") # 经过优化后的概念列表

In [ ]:
concepts_em[['概念_em','概念代码_em']]

In [ ]:
# !!!后续：更新续传问题

# 获取东方财富网概念信息
concepts_em = pd.read_csv('./行业概念/em_概念.csv',encoding="gbk") # 经过优化后的概念列表

concepts_original_em = ak.stock_board_concept_name_em()[['板块名称','板块代码']] # 东方财富原本的概念列表

all_stocks_concepts_em = pd.DataFrame()
for row, concept in concepts_em.iterrows():
    print('正在获取概念成分股： ' + concept['概念_em'], str(row+1) + '/'+str(len(concepts_em)) )
    stock_board_cons_em_df = ak.stock_board_concept_cons_em(symbol= \
                             concepts_original_em.loc[concepts_original_em['板块代码'] == concept['概念代码_em']]['板块名称'].values[0])
    #print(stock_board_cons_em_df)
    stock_board_cons_em_df['概念名称_em'] = concept['概念_em']
    stock_board_cons_em_df['概念代码_em'] = concept['概念代码_em']
    all_stocks_concepts_em = all_stocks_concepts_em.append(stock_board_cons_em_df)
print('over')    
    
#将同花顺概念分为 “行业”，“事件”和“交易”三类
industry_concepts_list_em = concepts_em.loc[concepts_em['概念种类'].isin(['行业'])]['概念_em'].values.tolist()
events_concepts_list_em = concepts_em.loc[concepts_em['概念种类'].isin(['事件','区域','知名企业'])]['概念_em'].values.tolist()
trading_concepts_list_em = concepts_em.loc[concepts_em['概念种类'].isin(['金融'])]['概念_em'].values.tolist()

#所有概念表格
all_concepts_stocks_em = all_stocks_concepts_em.groupby(['代码','名称'])['概念名称_em'].apply(lambda x: ",".join \
                            (list(set(x.str.cat(sep=',').split(','))))).reset_index()
#分别制作三类概念数据表格
industry_concepts_stocks_em = all_stocks_concepts_em.loc[all_stocks_concepts_em['概念名称_em'].isin(industry_concepts_list_em)]
industry_concepts_stocks_em = industry_concepts_stocks_em.groupby(['代码','名称'])['概念名称_em'].apply(lambda x: ",".join \
                            (list(set(x.str.cat(sep=',').split(','))))).reset_index().rename(columns={'概念名称_em': '行业概念_em'})

events_concepts_stocks_em = all_stocks_concepts_em.loc[all_stocks_concepts_em['概念名称_em'].isin(events_concepts_list_em)]
events_concepts_stocks_em = events_concepts_stocks_em.groupby(['代码','名称'])['概念名称_em'].apply(lambda x: ",".join \
                            (list(set(x.str.cat(sep=',').split(','))))).reset_index().rename(columns={'概念名称_em': '事件概念_em'})

trading_concepts_stocks_em = all_stocks_concepts_em.loc[all_stocks_concepts_em['概念名称_em'].isin(trading_concepts_list_em)]
trading_concepts_stocks_em = trading_concepts_stocks_em.groupby(['代码','名称'])['概念名称_em'].apply(lambda x: ",".join \
                            (list(set(x.str.cat(sep=',').split(','))))).reset_index().rename(columns={'概念名称_em': '交易概念_em'})

In [ ]:
# 通过tushare获取股票的省份和主营业务信息

import tushare as ts
pro = ts.pro_api('158ce95d6e799f55b8e8277aa1f6138fa71acf9c52df5ec667296fbc')
ts_stocks_info = pro.stock_company( fields='ts_code,chairman,manager,secretary,reg_capital,setup_date,province,city,employees,main_business,business_scope,introduction')

ts_stocks_info['ts_code'] = ts_stocks_info['ts_code'].str.replace('.SZ','').str.replace('.SH','').str.replace('.BJ','')
ts_stocks_info.columns = ['代码', '董事长', '总经理', '董事秘书', '注册资本', \
       '上市日期', '省份', '城市', '公司介绍', '营业范围', \
       '员工人数', '主营业务']

In [ ]:
import tushare as ts
pro = ts.pro_api('158ce95d6e799f55b8e8277aa1f6138fa71acf9c52df5ec667296fbc')
ts_stocks_info = pro.stock_company( fields='ts_code,chairman,manager,secretary,reg_capital,setup_date,province,city,employees,main_business,business_scope,introduction')

In [ ]:
ts_stocks_info.columns = ['股票代码', '董事长', '总经理', '董事秘书', '注册资本', \
       '上市日期', '省份', '城市', '公司介绍', '营业范围', \
       '员工人数', '主营业务']

In [ ]:
ts_stocks_info.to_csv('stock_basic_info.csv', index = False, encoding = 'gbk')

In [ ]:
# 所有股票的资产负债表
try:
    stock_zcfz_em = ak.stock_zcfz_em(date=str(today.year)+"1231")
except:
    try:
        stock_zcfz_em = ak.stock_zcfz_em(date=str(today.year)+"0930")
    except:
        try:
            stock_zcfz_em = ak.stock_zcfz_em(date=str(today.year)+"0630")
        except:
            stock_zcfz_em = ak.stock_zcfz_em(date=str(today.year)+"0331")

# 所有股票的利润表
try:
    stock_lrb_em = ak.stock_lrb_em(date=str(today.year)+"1231")
except:
    try:
        stock_lrb_em = ak.stock_lrb_em(date=str(today.year)+"0930")
    except:
        try:
            stock_lrb_em = ak.stock_lrb_em(date=str(today.year)+"0630")
        except:
            stock_lrb_em = ak.stock_lrb_em(date=str(today.year)+"0331")

In [ ]:
# # merge 行业和所有概念信息
all_stocks_info = all_stocks_ths_industry.merge(all_concepts_stocks_ths,how='outer',left_on=['代码','名称'],right_on=['代码','名称']) \
                                                .merge(industry_concepts_stocks_ths,how='outer',left_on=['代码','名称'],right_on=['代码','名称']) \
                                                .merge(events_concepts_stocks_ths,how='outer',left_on=['代码','名称'],right_on=['代码','名称']) \
                                                .merge(trading_concepts_stocks_ths,how='outer',left_on=['代码','名称'],right_on=['代码','名称']) \
                                                .merge(ts_stocks_info,how='outer',left_on=['代码'],right_on=['代码']) \
                                                .merge(stock_lrb_em,how='outer',left_on=['代码','名称'],right_on=['股票代码','股票简称']) \
                                                .merge(stock_zcfz_em,how='outer',left_on=['代码','名称'],right_on=['股票代码','股票简称'])

all_stocks_info.to_csv('all_stocks_info.csv')

In [ ]:
#最新的业绩报表
try:
    stock_yjbb_em_df = ak.stock_yjbb_em(date=str(today.year)+"1231")
except:
    try:
        stock_yjbb_em_df = ak.stock_yjbb_em(date=str(today.year)+"0930")
    except:
        try:
            stock_yjbb_em_df = ak.stock_yjbb_em(date=str(today.year)+"0630")
        except:
            stock_yjbb_em_df = ak.stock_yjbb_em(date=str(today.year)+"0331")
stock_yjbb_em_df

In [ ]:
#公告
stock_notice_report_df = ak.stock_notice_report(symbol='全部', date="20220728")
stock_notice_report_df

In [ ]:
stock_lhb_stock_statistic_em_df = ak.stock_lhb_stock_statistic_em(symbol="近一月")
stock_lhb_stock_statistic_em_df

In [ ]:
stock_lh_yyb_control_df = ak.stock_lh_yyb_control()
stock_lh_yyb_control_df

In [ ]:
stock_lh_yyb_control_df.to_csv('机构抱团.csv')

In [ ]:
stock_dzjy_hyyybtj_df = ak.stock_dzjy_hyyybtj(symbol='近3日')
stock_dzjy_hyyybtj_df

In [ ]:
# 问财-热门股票排名数据 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
stock_hot_rank_wc_df = ak.stock_hot_rank_wc(date="20220729")
stock_hot_rank_wc_df.loc[stock_hot_rank_wc_df['股票简称'] == '中复神鹰']

In [ ]:
stock_hot_rank_wc_df.to_csv('hottest.csv')

In [ ]:
stock_hot_keyword_em_df = ak.stock_hot_keyword_em(symbol="SH688295")
stock_hot_keyword_em_df

In [ ]:
#盘口异动数据
# symbol="大笔买入"; choice of {'火箭发射', '快速反弹', '大笔买入', '封涨停板', '打开跌停板', '有大买盘', '竞价上涨', '高开5日线', 
# '向上缺口', '60日新高', '60日大幅上涨', '加速下跌', '高台跳水', '大笔卖出', '封跌停板', '打开涨停板', '有大卖盘', '竞价下跌', '低开5日线', 
# '向下缺口', '60日新低', '60日大幅下跌'}
stock_changes_em_df = ak.stock_changes_em(symbol="大笔买入")
stock_changes_em_df

In [ ]:
stock_zh_a_alerts_cls_df = ak.stock_zh_a_alerts_cls()
stock_zh_a_alerts_cls_df

In [ ]:
stock_telegraph_cls_df = ak.stock_telegraph_cls()
stock_telegraph_cls_df

In [ ]:
stock_rank_ljqs_ths_df = ak.stock_rank_ljqs_ths()
stock_rank_ljqs_ths_df

In [ ]:
stock_rank_ljqs_ths_df.to_csv('量价齐升.csv')

In [ ]:
js_news_df = ak.js_news(timestamp="2022-07-29 00:57:18")
js_news_df

In [ ]:
stock_news_em_df = ak.stock_news_em(stock="688295")
stock_news_em_df

In [ ]:
car_cpca_energy_sale_df = ak.car_cpca_energy_sale()
car_cpca_energy_sale_df

In [ ]:
news_cctv_df = ak.news_cctv(date="20220728")
news_cctv_df

In [ ]:
stock_js_weibo_report_df = ak.stock_js_weibo_report(time_period="CNHOUR12")
stock_js_weibo_report_df